In [1]:
import numpy as np
from scipy import optimize as opt
import pandas as pd
from scipy.io import loadmat

In [2]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
X = np.matrix(data['X'])
X = np.insert(X, 0, 1, axis=1)
X.shape

(5000, 401)

In [5]:
m = X.shape[0]
n = X.shape[1]
K = 10
theta = np.matrix(np.zeros((K, n)))
Y = np.matrix(np.zeros((m, K)))
y = np.matrix(data['y'])

In [13]:
Y[:, 0] = np.matrix([1 if int(i)==10 else 0 for i in y]).T
for i in range(1, K):
    Y[:, i] = np.matrix([1 if int(j)==i else 0 for j in y]).T

In [14]:
# 得到了X,Y,theta
X.shape, Y.shape, theta.shape

((5000, 401), (5000, 10), (10, 401))

In [15]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
def costFunc(theta, X, y, _lambda):
    theta = np.matrix(theta)
    h_x = sigmoid(X*theta.T)
    normal = (1/m)*float(-y.T*np.log(h_x)-(1-y.T)*np.log(1-h_x))
    reg = (1/2*m)*float(theta*theta.T-theta[0,0]*theta[0,0]) # 注意不包括theta_0
    return reg+normal
costFunc(theta[0, :], X, Y[:, 0], 1)

0.693147180559947

In [20]:
def gradient(theta, X, y, _lambda):
    theta = np.matrix(theta)
    h_x = sigmoid(X*theta.T)
    err = h_x-y
    norm = (X.T*err).T/m
    reg = (_lambda*theta)/m
    reg[0, 0] = 0 # 注意不包括theta_0
    return norm+reg
gradient(theta[0, :], X, Y[:, 0], 1).shape

(1, 401)

In [23]:
# 使用theta, X, y, _lambda进行训练
def oneVSrest(theta, X, Y, _lambda):
    for i in range(K):
        theta_i = theta[i, :]
        y_i = Y[:, i]
        res = opt.minimize(fun=costFunc, 
                           x0=theta_i, 
                           args=(X, y_i, _lambda) , 
                           jac=gradient, 
                           method="TNC")
        theta[i, :] = np.matrix(res.x)
    return theta

In [24]:
_lambda = 1
res_theta = oneVSrest(theta, X, Y, _lambda)
res_theta.shape

C:\Users\RefrainKruso\AppData\Local\Temp\ipykernel_15936\2806270157.py:6: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = opt.minimize(fun=costFunc,


(10, 401)

In [30]:
# 利用res_theta进行分类
def classify(img, theta):
    img = np.matrix(np.reshape(img, (1, 400)))
    img = np.insert(img, 0, 1, axis=1)
    res = []
    for i in range(K):
        res.append(float(sigmoid(img*theta[i,:].T)))
    return res.index(max(res))

In [31]:
# 计算准确率
acc = 0
for i in range(m):
    img_i = X[i, 1:]
    prd = classify(img_i, theta)
    if Y[i, prd]==1:
        acc = acc + 1
print("accuracy="+str(acc*100/m)+"%")    

accuracy=75.5%
